In [1]:
import pandas as pd
from datetime import datetime
import sqlite3


In [2]:
df_objet_trouve = pd.read_csv("objet_perdus.csv")
df_gare = pd.read_csv("frequentations_gare.csv")
df_meteo = pd.read_csv("donneemeteoo.csv")

df_meteo['time'] = pd.to_datetime(df_meteo['time'])

# Regroupement par jour et calcul de la moyenne de la colonne "temperature_2m"
df_meteo= df_meteo.groupby(df_meteo['time'].dt.date)['temperature'].mean()
df_meteo = df_meteo.to_frame().reset_index()

In [3]:
connexion = sqlite3.connect("bddtest.db")
connexion.execute("""
CREATE TABLE IF NOT EXISTS gare (
    id_gare INTEGER PRIMARY KEY AUTOINCREMENT,
    nom_gare TEXT NOT NULL,
    code_postal INTEGER,
    frequentation_2019 INTEGER,
    frequentation_2020 INTEGER,
    frequentation_2021 INTEGER,
    frequentation_2022 INTEGER
)
""")

connexion.execute("""
CREATE TABLE IF NOT EXISTS objet_trouve (
    id_objet INTEGER PRIMARY KEY AUTOINCREMENT,
    nom_gare TEXT NOT NULL,
    type_objet TEXT NOT NULL,
    nature_objet TEXT NOT NULL,
    date TEXT NOT NULL,
    FOREIGN KEY (nom_gare) REFERENCES gare(nom_gare)

)
""")

connexion.execute("""
CREATE TABLE IF NOT EXISTS meteo (
    id_meteo INTEGER PRIMARY KEY AUTOINCREMENT,
    time TEXT NOT NULL,
    temperature REAL NOT NULL
)
""")

for index, row in df_gare.iterrows():
    connexion.execute(
        "INSERT INTO gare (nom_gare, code_postal, frequentation_2019, frequentation_2020, frequentation_2021, frequentation_2022) VALUES (?, ?, ?, ?, ?, ?)",
        (row["nom_gare"], row["code_postal"], row["frequentation_2019"], row["frequentation_2020"], row["frequentation_2021"], row["frequentation_2022"])
    )

for index, row in df_objet_trouve.iterrows():
    connexion.execute(
        "INSERT INTO objet_trouve (nom_gare, type_objet, nature_objet, date) VALUES (?, ?, ?, ?)",
        (row["Nom gare"], row["Type d'objet"], row["Nature de l'objet"], row["Date"])
    )

for index, row in df_meteo.iterrows():
    connexion.execute(
        "INSERT INTO meteo (time, temperature) VALUES (?, ?)",
        (row["time"], row["temperature"])
    )
    
connexion.commit()
connexion.close()